In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import random as rnd

# Load Data

In [2]:
def load_data(file, sep='\t'):
    return pd.read_csv(f'./lfm-challenge-data/{file}', delimiter=sep)

In [3]:
users = load_data('lfm-challenge.user')
items = load_data('lfm-challenge.item')
inter_train = load_data('lfm-challenge.inter_train')
inter_test = load_data('lfm-challenge.inter_test')
test_users = pd.read_csv(f'./lfm-challenge-data/test_indices.txt')['users'].values

n_users = users['user_id'].values.size
n_items = items.index.values.size

In [4]:
users

,user_id,country,age_at_registration,gender,registration_date
0,0,RU,27,m,2007-03-27 19:50:20
1,1,IT,33,m,2006-06-18 21:07:33
2,2,BR,19,m,2010-01-14 05:55:11
3,3,RU,25,m,2007-10-12 18:42:00
4,4,UK,25,m,2005-06-15 22:02:11
...,...,...,...,...,...
9694,9694,RU,24,m,2010-10-25 00:55:47
9695,9695,US,24,f,2005-04-15 04:29:35
9696,9696,RU,20,m,2011-07-29 21:09:24
9697,9697,DE,18,m,2010-05-14 14:07:05


# Interaction Matrix

In [5]:
def create_interaction_matrix(users, items, inter, threshold=1, binary=False):
    interaction_matrix = np.zeros((n_users, n_items), dtype=np.int8)
    
    for user in range(n_users):
        interacted_items = inter.loc[inter['user_id'] == user, 'item_id'].values
        rate_of_items = inter.loc[inter['user_id'] == user, 'listening_events'].values
        
        for item in range(interacted_items.size):
            rating = rate_of_items[item]
            if binary:
                rating = 0 if rating < threshold else 1
            
            interaction_matrix[user, interacted_items[item]] = rating
    
    return interaction_matrix

In [6]:
interaction_matrix = create_interaction_matrix(users, items, inter_train, binary=True)
test_interaction_matrix = create_interaction_matrix(users, items, inter_test, binary=True)

## Test Predictions

In [12]:
def get_ndcg_score(predictions: np.ndarray, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    predictions - np.ndarray - predictions of the recommendation algorithm for each user.
    test_interaction_matrix - np.ndarray - test interaction matrix for each user.
    topK - int - topK recommendations should be evaluated.
    
    returns - average ndcg score over all users.
    """
    n_users = predictions.shape[0]
    discounts = np.log2(np.arange(2, topK+2)) # discounts for positions 1 to topK (0-indexed)
    ndcg_scores = np.zeros(n_users)

    for user in range(n_users):
        top_items = predictions[user]
        relevant_items = test_interaction_matrix[user].nonzero()[0]
        if len(relevant_items) == 0:
            continue

        # calculate DCG
        dcg = 0
        for i, item in enumerate(top_items):
            if item in relevant_items:
                dcg += 1 / discounts[i]

        # calculate IDCG
        n_relevant = min(topK, len(relevant_items))
        idcg = np.sum(1 / discounts[:n_relevant]) # it is 1 for each relevant item in ideal case
        
        ndcg_scores[user] = dcg / idcg

    return np.mean(ndcg_scores)


In [13]:
predictions = np.load('itemknn_rec.npy')
predictions

array([[7258,  148, 3432, 6904, 7767,  970,  157, 1518,  152,  271],
       [3015, 2349,  170, 5423, 8566, 5190, 3394, 2707, 6955, 4144],
       [1999, 3580, 2011, 9302,  410, 3999, 5210, 8126, 3174, 8167],
       [ 986, 3867, 7682, 4293, 7038, 3490, 6921, 3717,  255, 1561],
       [1358,  351, 9408, 1263,  203,  345, 1274, 1273, 9410, 1189],
       [ 593,  114,   56,  383, 2359,   86,  400, 2605,   68,  152],
       [5941,  458, 7802, 6212, 8773, 8320, 5666,  710, 2366, 5820],
       [ 410,  420,  449,  415,  728, 2359, 2203,  593, 2605,  928],
       [7580, 9391, 1697, 5086, 9702,  104,   88, 7846, 2664, 8259],
       [7700, 8364, 1526, 1556,  897, 4914,  649, 6151,  352, 5682],
       [ 502,  251, 8664,  231,  239, 5340, 1545,  298, 5342,    2],
       [ 157, 1509,  313,  148,  368, 1134, 3305,  160, 1540, 4144],
       [5286, 2282, 5324, 5330, 2214, 5328,  157, 7513, 1305, 3808],
       [1069, 8607, 3194,  669, 3063, 2963, 7136, 8478,  647, 3173],
       [3173, 4094, 1851, 1040, 10

In [14]:
score = get_ndcg_score(predictions, test_interaction_matrix)
score

0.0013886244387355457

# POP Recommender Country

In [70]:
def recTopKPopByCountry(inter_matr: np.array,
               user: int,
               top_k: int,
               users: pd.DataFrame) -> np.array:
    '''
    inter_matr - np.array from the task 1;
    user - user_id, integer;
    top_k - expected length of the resulting list;
    users: pandas Dataframe consisting of user information for all users, requires a "country" column

    returns - list/array of top K popular items that the user has never seen
              (sorted in the order of descending popularity);
    '''
    # TODO: YOUR IMPLEMENTATION
    top_pop = None

    rec_items_before = np.where(inter_matr[user] == 1)[0]
    age_at_registration = users.loc[users['user_id'] == user, 'age_at_registration'].values[0]
    users_from_same_age_at_registration = users.loc[users['age_at_registration'] == age_at_registration, 'user_id'].values
    new_inter_matr = inter_matr[users_from_same_age_at_registration]
    
    interactions = pd.Series(np.sum(new_inter_matr, axis=0)).drop(rec_items_before).sort_values(ascending=False, kind='mergesort')
    top_pop = interactions.index[:top_k].to_numpy()
    
    return top_pop

In [72]:
predictions = []

for user in range(n_users):
    print(f'User {user}/{n_users}', end='\r')
    top_10 = recTopKPopByCountry(inter_matr=interaction_matrix, user=user, top_k=10, users=users)
    predictions.append(top_10)

predictions = np.array(predictions)

In [73]:
predictions.shape

(9699, 10)

In [74]:
score = get_ndcg_score(predictions, test_interaction_matrix)
score

0.027594118519486035

In [14]:
inter_train

,user_id,item_id,listening_events
0,0,3,4
1,0,4,2
2,0,6,2
3,0,7,3
4,1,11,3
...,...,...,...
138767,9698,720,7
138768,9698,727,3
138769,9698,2586,2
138770,9698,2963,2


# Neural Network

In [15]:
class MF(nn.Module):

    def __init__(self, n_users: int, n_items: int, n_factors: int):
        """
        n_users - int - number of users.
        n_items - int - number of items.
        n_factors - int - dimensionality of the latent space.
        """

        super(MF, self).__init__()

        self.embedding_user = nn.Embedding(n_users, n_factors)
        self.embedding_item = nn.Embedding(n_items, n_factors)
        
        # TODO: YOUR IMPLEMENTATION.


    def forward(self, user: torch.Tensor, item: torch.Tensor) -> torch.Tensor:
        """
        We allow for some flexibility giving lists of ids as inputs:
        if the training data is small we can deal with it in a single forward pass,
        otherwise we could fall back to mini-batches, limiting users and items we pass
        every time.
        
        user - torch.Tensor - user_ids.
        item - torch.Tensor - item_ids.
        
        returns - torch.Tensor - Reconstructed Interaction matrix of shape (n_users, n_items).
        """
        u = self.embedding_user(user)
        v = self.embedding_item(item)

        # TODO: YOUR IMPLEMENTATION.
        
        return (u @ v.T)

In [16]:
def compute_loss(logits: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
    """
    logits - torch.Tensor - output of model.
    labels - torch.Tensor - labels / interaction matrix model should learn to reconstruct.
    
    returns - torch.Tensor - BCELoss over all logits and labels.
    """
    loss = nn.BCELoss()(torch.sigmoid(logits), labels)
    
    # TODO: YOUR IMPLEMENTATION.

    return loss

In [17]:
def train(model: nn.Module, train_data_inter: np.ndarray, epochs: int, optimizer, loss_func) -> list:
    """
    model - nn.Module - torch module to train.
    train_data_inter - np.ndarray - interaction matrix of the training data.
    epochs - int - number of epochs to perform.
    optimizer - optim - optimizer for training.
    loss_func - loss function for training.
    
    returns - list - list of loss values over all epochs.
    """
    losses = []

    model.train()

    user_ids = torch.Tensor(list(range(train_data_inter.shape[0]))).long()
    item_ids = torch.Tensor(list(range(train_data_inter.shape[1]))).long()
    y = torch.Tensor(train_data_inter).long()

    for e in range(epochs):
        optimizer.zero_grad()

        y_hat = model(user_ids, item_ids)

        loss = loss_func(y_hat.unsqueeze(0).float(), y.unsqueeze(0).float())
        loss.backward()
        optimizer.step()

        if e % 100 == 0:
            print("Loss ", e, ": ", loss.item())

        losses.append(loss.item())

    return losses

In [18]:
# Do not change the seed.
torch.manual_seed(1234)
rnd.seed(1234)
np.random.seed(1234)

model_128 = MF(n_users=n_users, n_items=n_items, n_factors=128)
optimizer = optim.Adam(model_128.parameters(), lr=0.001)

# TODO: YOUR IMPLEMENATION.
# Initialize the model and optimizer as prescribed


In [19]:
loss_model_128 = train(model=model_128,
                       train_data_inter=interaction_matrix,
                       epochs=1000,
                       optimizer=optimizer,
                       loss_func=compute_loss)

assert len(loss_model_128) == 1000, "Loss should have 1000 elements, one for each epoch."

Loss  0 :  10.083086013793945
Loss  100 :  7.214507579803467
Loss  200 :  3.5739328861236572
Loss  300 :  0.8108418583869934
Loss  400 :  0.21792972087860107
Loss  500 :  0.09922955930233002
Loss  600 :  0.064073845744133
Loss  700 :  0.050044916570186615
Loss  800 :  0.04316549748182297
Loss  900 :  0.039255231618881226


In [20]:
def itMF_recommend(user_id: int, seen_item_ids: list, model=None, topK=10) -> np.ndarray:
    """
    Recommend with the trained model to selected users
    
    user_id - int - id of target user.
    seen_item_ids - list[list[int]] ids of items already seen by the users (to exclude from recommendation)
    model - trainted factorization model to use for scoring
    topK - number of recommendations per user to be returned
    
    returns - np.ndarray - np.ndarray - list of ids of recommended items in the order of descending score
                           use -1 as a place holder item index, when it is impossible to recommend topK items
    """
    recs = None

    model.eval()
    user_embedding = model.embedding_user(torch.tensor([user_id]))
    item_embeddings = model.embedding_item.weight
    
    # Compute the score for each item
    all_ratings = torch.matmul(user_embedding, item_embeddings.T).flatten()
    
    # Set scores of seen items to -inf to ignore them
    all_ratings[seen_item_ids] = float('-inf')
    
    # Get the indices of the top K items
    _, top_indices = torch.topk(all_ratings, topK)
    recs = top_indices.tolist()

    return np.array(recs)

In [22]:
user_id = 5
seen_list = np.where(interaction_matrix[user_id] != 0)

itMF_recommend(user_id, seen_list, model_128)

array([2401, 5887, 1043, 8751, 4345, 8473, 1708, 5059, 5805, 4029])

In [25]:
def recommend():
    recommendations = []
    
    for user_id in range(n_users):
        print(f'{user_id}/{n_users}', end='\r')
        seen_list = np.where(interaction_matrix[user_id] != 0)
        rec = itMF_recommend(user_id, seen_list, model_128)
        recommendations.append(rec)
    
    return np.array(recommendations)

In [26]:
recs = recommend()

In [35]:
recs.shape

(9699, 10)

In [39]:
a = recs[test_users]
b = test_interaction_matrix[test_users]
b.shape

(100, 10000)

In [40]:
score = get_ndcg_score(a, b)
score

0.0010778915452451094